## CAPSTONE Project Course  - Data Analysis - Week 1

This notebook will be mainly used for Capstone project

In [70]:
import pandas as pd
import numpy as np

In [71]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [72]:
path='https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
df = pd.read_csv(path)

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
df_coll = df[['WEATHER','ROADCOND','SEVERITYCODE']]
df_coll.shape

(194673, 3)

In [74]:
# Check how many records have either WEATHER or ROADCOND values as null
print("Total number of records with null values :",df_coll[(df_coll['WEATHER'].isnull()&df_coll['ROADCOND'].isnull())].shape)

# Drop those null records 
df_coll.dropna(subset=['WEATHER','ROADCOND'],axis = 0, how='any',inplace=True)
print("After deleting the null records",df_coll.shape)

Total number of records with null values : (4963, 3)
After deleting the null records (189543, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
# Check the influence of type of WEATHER on the SEVERITY of the collision
df_weath = pd.concat([df_coll['WEATHER'],pd.get_dummies(df_coll["SEVERITYCODE"])], axis=1)
df_weath.rename(columns={1:'SEV1', 2:'SEV2'}, inplace=True)
df_weath = df_weath.groupby(['WEATHER'],as_index=True).sum()

# Visualize the data in percentages for a better view
df_weath[['SEV1','SEV2']] = df_weath[['SEV1','SEV2']].div(df_weath[['SEV1','SEV2']].sum(axis=1), axis=0).multiply(100).round(decimals=2).astype(str) + '%'
df_weath

,SEV1,SEV2
WEATHER,,
Blowing Sand/Dirt,72.73%,27.27%
Clear,67.75%,32.25%
Fog/Smog/Smoke,67.14%,32.86%
Other,86.02%,13.98%
Overcast,68.44%,31.56%
Partly Cloudy,40.0%,60.0%
Raining,66.28%,33.72%
Severe Crosswind,72.0%,28.0%
Sleet/Hail/Freezing Rain,75.22%,24.78%


From the above view, weather conditions can be grouped into 4 bins based on the ratio of high severity collissions  
- SEV2 >35% can be classifiend as "RISKYWEATHER"  
- SEV2 >30% and <35% can be classifiend as "BADWEATHER"  
- SEV2 >20% abd <30% can be classifiend as "CAUTIONWEATHER"  
- SEV2 <20% can be classifiend as "GOODWEATHER"  

In [76]:
# Check the influence of type of ROADCOND on the SEVERITY of the collision
df_road = pd.concat([df_coll['ROADCOND'],pd.get_dummies(df_coll["SEVERITYCODE"])], axis=1)
df_road.rename(columns={1:'SEV1', 2:'SEV2'}, inplace=True)
df_road = df_road.groupby(['ROADCOND'],as_index=True).sum()

# Look at the data in percentages for a better view
df_road[['SEV1','SEV2']] = df_road[['SEV1','SEV2']].div(df_road[['SEV1','SEV2']].sum(axis=1), axis=0).multiply(100).round(decimals=2).astype(str) + '%'
df_road

,SEV1,SEV2
ROADCOND,,
Dry,67.83%,32.17%
Ice,77.36%,22.64%
Oil,62.5%,37.5%
Other,67.42%,32.58%
Sand/Mud/Dirt,68.92%,31.08%
Snow/Slush,83.43%,16.57%
Standing Water,73.91%,26.09%
Unknown,95.03%,4.97%
Wet,66.81%,33.19%


From the above view, ROAD conditions can be grouped into 4 bins based on the ratio of high severity collissions  
- SEV2 >35% can be classifiend as "RISKYROAD"  
- SEV2 >20% and <35% can be classifiend as "BADROAD"  
- SEV2 >10% abd <20% can be classifiend as "CAUTIONROAD"  
- SEV2 <20% can be classifiend as "GOODROAD" 

In [77]:
# Prepare the Data
Features = df_coll[['SEVERITYCODE']].copy()

# Convert WEATHER conditions to columns
W_Features =  pd.get_dummies(df_coll['WEATHER'])

# Map WEATHER conditions to Features bins
Features['RISKYWEATHER']=W_Features['Partly Cloudy']
Features['BADWEATHER']=W_Features['Clear']+W_Features['Fog/Smog/Smoke']+W_Features['Overcast']+W_Features['Raining']
Features['CAUTIONWEATHER']=W_Features['Blowing Sand/Dirt']+W_Features['Sleet/Hail/Freezing Rain']+W_Features['Severe Crosswind']
Features['GOODWEATHER']=W_Features['Snowing']+W_Features['Unknown']+W_Features['Other']

# Convert ROAD conditions to columns
R_Features =  pd.get_dummies(df_coll['ROADCOND'])

# Map ROADCOND conditions to Features bins
Features['RISKYROAD']=R_Features['Oil']
Features['BADROAD']=R_Features['Dry']+R_Features['Sand/Mud/Dirt']+R_Features['Wet']+R_Features['Other']
Features['CAUTIONROAD']=R_Features['Standing Water']+R_Features['Ice']
Features['GOODROAD']=R_Features['Snow/Slush']+R_Features['Unknown']

Features.head()

,SEVERITYCODE,RISKYWEATHER,BADWEATHER,CAUTIONWEATHER,GOODWEATHER,RISKYROAD,BADROAD,CAUTIONROAD,GOODROAD
0,2,0,1,0,0,0,1,0,0
1,1,0,1,0,0,0,1,0,0
2,1,0,1,0,0,0,1,0,0
3,1,0,1,0,0,0,1,0,0
4,2,0,1,0,0,0,1,0,0
